In [ ]:
import pdb
import pycm
import sys
sys.path.append('../')
from glob import glob
import torch
import scipy
from torch import nn
import cv2
from tqdm import tqdm_notebook as tqdm
import os
# from models import Model
from PIL import Image
from skimage import io, transform
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from torchvision.datasets.folder import pil_loader
import random
import numpy as np
from torchsummary import summary
import pretrainedmodels
from sklearn.model_selection import KFold, StratifiedKFold
from efficientnet_pytorch import EfficientNet
import albumentations
from pycm import ConfusionMatrix
from tensorboard_logger import * 
import warnings
from dataloader import provider
warnings.filterwarnings("ignore")
from models import *
from image_utils import *
import albumentations
from albumentations import torch as AT
os.environ['CUDA_VISIBLE_DEVICES']="" # useful when testing dataloader
%matplotlib inline


In [ ]:
from torchvision import models
resnet = models.resnet50()

beginning = nn.Sequential(*list(resnet.children())[:-3])

end = nn.Sequential(*list(resnet.children())[-3:])

full = nn.Sequential(beginning, end)

a = torch.rand((2, 3, 256, 256), device=torch.device("cpu"))

#full(a)

In [ ]:
out = beginning(a)

In [ ]:
out.shape

### PyCM testing

In [ ]:
y_actual = [1, 2, 3, 4]
y_pred = [1, 1, 3, 2]
cm = ConfusionMatrix(y_actual, y_pred)

In [ ]:
cm.qwk  = 1

In [ ]:
cm.save_obj('test.obj', save_stat=True, save_vector=True)

In [ ]:
cm_loaded = ConfusionMatrix(file=open("test.obj.obj"))

In [ ]:
cm_loaded.qwk


In [ ]:
test = open('test.obj.obj')
red = test.read()

In [ ]:
import json
red = json.loads(red)

In [ ]:
red['qwk']

ConfusionMatrix is not saving additional attributes, let's modify those which are already present

In [ ]:

class CM(ConfusionMatrix):

    def __init__(
            self,
            *args):
        ConfusionMatrix.__init__(self,
            *args)
        
    def save(self,
             name,
             qwk=None,
             loss=None, 
             **kwargs):
        status = self.save_obj(name, **kwargs)
        obj_full_path = status['Message']
        print(obj_full_path)
        with open(obj_full_path, 'r') as f:
            dump_dict = json.load(f)
            dump_dict['Overall-Stat']['qwk'] = qwk
            dump_dict['Overall-Stat']['loss'] = loss
        json.dump(dump_dict, open(obj_full_path, 'w'))
 

In [ ]:
y_actual = [1, 2, 3, 4]
y_pred = [1, 1, 3, 2]
cm = CM(y_actual, y_pred)

In [ ]:
cm.save('test', qwk=1, loss=1, save_stat=True, save_vector=True)

In [ ]:
f = open('test.obj', 'r')
data = json.load(f)

In [ ]:
data

In [ ]:
data['qwk']

In [ ]:

cm_loaded = ConfusionMatrix(file=open("test.obj"))

In [ ]:
cm_loaded.overall_stat

It's has got some internal function which needs to be overwritten

In [ ]:
cm.ERR  = 0

In [ ]:

cm.save_obj('test', save_stat=True, save_vector=True)

In [ ]:
cm_loaded = ConfusionMatrix(file=open("test.obj"))

In [ ]:
cm_loaded.ERR

Can't modify the values like this, it recalculates before saving

### Rank averaging

In [ ]:
# START
files = {}
files['file1'] = '../weights/submission785.csv'
files['file2'] = '../weights/submission796.csv'
files['file3'] = '../weights/submission77.csv'
# END


dfs = []
for key, _file in files.items():
    df = pd.read_csv(_file)
    df.rename(columns = {'diagnosis': key}, inplace=True)
    dfs.append(df)



_submission = pd.concat(dfs, axis=1)



In [ ]:
_submission.head()

In [ ]:
a = _submission['file1']
a.rank?

In [ ]:

for key in files.keys():
    _submission[key + '_rank'] = _submission[key].rank()



In [ ]:
_submission.head()

In [ ]:

_submission['rank_sum'] = np.sum(
        _submission[col] for col in _submission.columns if '_rank' in col)
_submission['target'] = _submission['rank_sum']/(len(files) *
        _submission.shape[0])

# take the first (id) and last column (target)
submission = _submission.iloc[:, [0, -1]]

In [ ]:
_submission.head()

### Tesing EfficientNet

In [ ]:
#torch.hub.load?

In [ ]:
model = EfficientNet.from_name('efficientnet-b7')

In [ ]:
EfficientNet.from_name?

In [ ]:
summary(model, input_size=(3, 300, 300), device="cpu")

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=1)


In [ ]:
def pytorch_count_params(model):
    from functools import reduce    
    "count number trainable parameters in a pytorch model"
    total_params = sum(reduce( lambda a, b: a*b, x.size()) for x in model.parameters())
    return total_params

In [ ]:
pytorch_count_params(model)

In [ ]:
model

In [ ]:
a = 'asdf1234'

In [ ]:
a.startswith('asd')

In [ ]:
model = get_model('se_resnet50_v0')

In [ ]:
model.model.features

### test

In [ ]:
df = pd.read_csv('../data/train_all.csv')

In [ ]:
df.shape

In [ ]:
df_mod = df.iloc[:5000]

In [ ]:
df_mod.shape

### Activation maps, heatmaps, forward hooks, base model features

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt


class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 3, 1, 1)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(3, 6, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2)
        
        self.conv_trans1 = nn.ConvTranspose2d(6, 3, 4, 2, 1)
        self.conv_trans2 = nn.ConvTranspose2d(3, 1, 4, 2, 1)
        
    def forward(self, x):
        x = F.relu(self.pool1(self.conv1(x)))
        x = F.relu(self.pool2(self.conv2(x)))        
        x = F.relu(self.conv_trans1(x))
        x = self.conv_trans2(x)
        return x

dataset = datasets.MNIST(
    root='PATH',
    transform=transforms.ToTensor(),
    download=True
)
loader = DataLoader(
    dataset,
    num_workers=2,
    batch_size=8,
    shuffle=True
)

model = MyModel()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 1
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data)
        loss.backward()
        optimizer.step()
        
        print('Epoch {}, Batch idx {}, loss {}'.format(
            epoch, batch_idx, loss.item()))


def normalize_output(img):
    img = img - img.min()
    img = img / img.max()
    return img



In [ ]:
output.shape, pred.shape

In [ ]:
# Plot some images
idx = torch.randint(0, output.size(0), ())
pred = normalize_output(output[idx, 0])
img = data[idx, 0]

fig, axarr = plt.subplots(1, 2)
axarr[0].imshow(img.detach().numpy())
axarr[1].imshow(pred.detach().numpy())


In [ ]:

# Visualize feature maps
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.conv1.register_forward_hook(get_activation('conv1'))


In [ ]:

data, _ = dataset[0]
data.unsqueeze_(0)
output = model(data)

act = activation['conv1'].squeeze()


In [ ]:
data.shape, output.shape, act.shape

In [ ]:
fig, axarr = plt.subplots(act.size(0))
for idx in range(act.size(0)):
    axarr[idx].imshow(act[idx])

In [ ]:
model_name = "densenet121"
model = Model(model_name, 5)

In [ ]:
model.model.features.norm5.register_forward_hook(get_activation('norm5'))

In [ ]:
#model.model.features.register_forward_hook()

In [ ]:
img = torch.rand(1, 3, 224, 224)
output = model(img)
img.shape, output.shape

In [ ]:
act = activation['norm5'].squeeze()

In [ ]:
act.shape

In [ ]:
avg_act = torch.mean(act, 0)

In [ ]:

plt.imshow(avg_act)

In [ ]:
scipy.optimize.minimize?

In [ ]:
arr = np.array([[1, 1, 1, 1, 1], [1, 1, 0, 0, 0], [1, 0, 1, 1, 0], [0, 0, 0, 0, 0]])
mask = arr == 0
np.clip(np.where(mask.any(1), mask.argmax(1), 5) - 1, 0, 4)

In [ ]:
mask

In [ ]:
mask.any(1), mask.argmax(1) 

In [ ]:
np.where(mask.any(1), mask.argmax(1), 5)

In [ ]:
np.where?

In [ ]:
target = [1, 2, 3, 4, 0, 1, 2, 3, 4]
NUM_CLASSES = 5
to_multi_label(target)

###  pycm analysis

In [ ]:
from pycm import *
y_actu = [2, 0, 2, 2, 0, 1, 1, 2, 2, 0, 1, 2] # or y_actu = numpy.array([2, 0, 2, 2, 0, 1, 1, 2, 2, 0, 1, 2])
y_pred = [0, 0, 2, 1, 0, 2, 1, 0, 2, 0, 2, 2] # or y_pred = numpy.array([0, 0, 2, 1, 0, 2, 1, 0, 2, 0, 2, 2])
cm = ConfusionMatrix(actual_vector=y_actu, predict_vector=y_pred) # Create CM From Data

In [ ]:
cm.ACC

In [ ]:
cm.stat()

In [ ]:
!ls

In [ ]:
cms = {}
cmfiles = glob('weights/5Jul_resnext101_32x4d_fold1/logs/*val*.obj')
for i in range(len(cmfiles)):
    cms[f"cm{i}"] = ConfusionMatrix(file=open(f"weights/5Jul_resnext101_32x4d_fold1/logs/cmval_{i}.obj","r"))

In [ ]:
cp = Compare(cms)

In [ ]:
print(cp)

In [ ]:
cms['cm0'].overall_stat

In [ ]:
save_folder = "weights"
phase = "val"
val_logger = Logger(os.path.join(save_folder, "logs/val"),)
train_logger = Logger(os.path.join(save_folder, "logs/train"))
#configure( flush_secs=5)

In [ ]:
metrics = ["Overall ACC", "Kappa", "TPR Micro", "PPV Micro", "F1 Micro", "Cross Entropy", ]
overall_stats = {x:[] for x in metrics}
for i in range(len(cmfiles)):
    for j in overall_stats.keys():
        overall_stats[j].append(cms["cm%d" % i].overall_stat[j])

In [ ]:
def log_all(name, data_list, logger):
    for idx, val in enumerate(data_list):
        logger.log_value(name, val, idx)

In [ ]:
for metric in metrics:
    log_all(metric, overall_stats[metric], val_logger)

In [ ]:
a = torch.Tensor([[0, 0, 1], [1, 0, 0]])

In [ ]:
torch.argmax(a, dim=1)

In [ ]:
torch.argmax?

In [ ]:
labels = [1, 0, 2, 3, 2, 3, 3, 3]
#np.eye(len(labels), 4)
#np.eye(4)[1]
len(np.unique(labels))kk

In [ ]:
!ls data

In [ ]:
train_df = pd.read_csv('data/train_labels.csv')
test_df = pd.read_csv('data/sample_submission.csv')

In [ ]:
test_df.shape

In [ ]:
train_df.head()

In [ ]:
train_df.at[0, 'label'] = 1

In [ ]:
kfold = StratifiedKFold(20, shuffle=True, random_state=69)

In [ ]:
kfold.split(train_df['id'], train_df['label'])

In [ ]:
train_df.iloc[test]

In [ ]:
train_df.head()

In [ ]:
train_df.info(), test_df.info()

In [ ]:
%%time
img = mpimg.imread('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif')

In [ ]:
timeit mpimg.imread('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif')

In [ ]:
timeit pil_loader('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif')

In [ ]:
timeit cv2.imread('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif', flags=cv2.IMREAD_COLOR)

In [ ]:
timeit io.imread("data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif")

In [ ]:
img = torch.Tensor(img)

In [ ]:
img.flip?

In [ ]:
torch.flip?

### use cv2.imread for reading images 

In [ ]:
img.shape


In [ ]:
plt.imshow(img)

In [ ]:
# plot some random training images with their ground truth labels
i = random.choice(range(train_df.shape[0]))
img = mpimg.imread('data/train/' + train_df.iloc[i]['id'] + '.tif')
plt.imshow(img)
plt.title(train_df.iloc[i]['label'])
plt.show()

In [ ]:
train_df['label'].value_counts()

In [ ]:
train_df['label'].hist

In [ ]:
img = mpimg.imread('data/train/dd6dfed324f9fcb6f93f46f32fc800f2ec196be2.tif')
plt.imshow(img)
plt.show();
aug_img = train_aug(image=img)
plt.imshow(aug_img['image'])
plt.show();

### Dataloader analysis


In [ ]:

def resampled(df):
    ''' resample from df with replace=False'''
    def sample(obj):  # [5]
        return obj.sample(n=count_dict[obj.name], replace=False)

    count_dict = {
        0: 8000,
        2: 5292,
        1: 2443,
        3: 873,
        4: 708,
    }
        
    sampled_df = df.groupby('diagnosis').apply(sample).reset_index(drop=True)

    return sampled_df



In [ ]:
df = pd.read_csv('../data/train_old.csv')
df = resampled(df)

kfold = StratifiedKFold(7, shuffle=True, random_state=69)
train_idx, val_idx = list(kfold.split(
    df["id_code"], df["diagnosis"]))[1]
train_df, val_df = df.iloc[train_idx], df.iloc[val_idx]



In [ ]:
[x for x in train_df.id_code.tolist() if x in val_df.id_code.tolist()]

In [ ]:
from torch.utils.data import DataLoader, Dataset, sampler

class ImageDataset(Dataset):
    """training dataset."""

    def __init__(self, df, phase, cfg):
        """
        Args:
                fold: for k fold CV
                images_folder: the folder which contains the images
                df_path: data frame path, which contains image ids
                transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.phase = phase
        self.df = df
        self.num_samples = self.df.shape[0]
        self.fnames = self.df["id_code"].values
        self.labels = self.df["diagnosis"].values.astype("int64")
        self.num_classes = len(np.unique(self.labels))
        # self.labels = to_multi_label(self.labels, self.num_classes)  # [1]
        # self.labels = np.eye(self.num_classes)[self.labels]
        self.transform = get_transforms(phase, cfg)
        self.root = cfg['data_folder']

        '''
        self.images = []
        for fname in tqdm(self.fnames):
            path = os.path.join(self.images_folder, "bgcc300", fname + ".npy")
            image = np.load(path)
            self.images.append(image)
        '''

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        label = self.labels[idx]
        path = os.path.join(self.root, fname + ".npy")
        image = np.load(path)
        image = image.astype('uint8')
        #print(np.unique(image, return_counts=True))
        #image = self.images[idx]
        image = self.transform(image=image)["image"]
        return fname, image, label

    def __len__(self):
        # return 100
        return len(self.df)


def get_transforms(phase, cfg):
    size = cfg['size']
    mean = eval(cfg['mean'])
    std = eval(cfg['std'])
    print(type(size), type(mean), type(std))
    list_transforms = [
        # albumentations.Resize(size, size) # now doing this in __getitem__()
    ]
    if phase == "train":
        list_transforms.extend(
            [
                albumentations.Transpose(p=0.5),
                albumentations.Flip(p=0.5),
                albumentations.ShiftScaleRotate(
                    shift_limit=0,  # no resizing
                    scale_limit=0.1,
                    rotate_limit=120,
                    p=0.5,
                    border_mode=cv2.BORDER_CONSTANT
                ),
                albumentations.RandomBrightnessContrast(p=0.25),
            ]
        )
    list_transforms.extend(
        [

            albumentations.Normalize(mean=mean, std=std, p=1),
            #albumentations.Resize(size, size),
            AT.ToTensor(normalize=None),  # [6]
        ]
    )
    return albumentations.Compose(list_transforms)


def get_sampler(df, cfg):
    class_weights = cfg['class_weights']
    print("weights", class_weights)
    dataset_weights = [class_weights[idx] for idx in df["diagnosis"]]
    #dataset_weights = df["weight"].values
    datasampler = sampler.WeightedRandomSampler(dataset_weights, len(df))

    return datasampler


def resampled(df):
    ''' resample `total` data points from old data, following the dist of org data '''
    def sample(obj):  # [5]
        return obj.sample(n=count_dict[obj.name], replace=False)

    count_dict = {
        0: 6000,
        2: 5292,
        1: 2443,
        3: 873,
        4: 708
    }  # notice the order of keys

    sampled_df = df.groupby('diagnosis').apply(
        sample).reset_index(drop=True)

    return sampled_df


def provider(phase, cfg):
    fold = cfg['fold']
    total_folds = cfg['total_folds']
    data_folder = cfg['data_folder']
    df_path = cfg['df_path']
    class_weights = eval(cfg['class_weights'])
    batch_size = cfg['batch_size'][phase]
    num_workers = cfg['num_workers']
    num_samples = cfg['num_samples']

    df = pd.read_csv(df_path)
    kfold = StratifiedKFold(total_folds, shuffle=True, random_state=69)
    train_idx, val_idx = list(kfold.split(
        df["id_code"], df["diagnosis"]))[fold]
    train_df, val_df = df.iloc[train_idx], df.iloc[val_idx]

    df = train_df if phase == "train" else val_df
    image_dataset = ImageDataset(df, phase, cfg)

    datasampler = None
    if phase == "train" and class_weights:
        datasampler = get_sampler(df, cfg)
    print('datasampler:', datasampler)

    dataloader = DataLoader(
        image_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        shuffle=False if datasampler else True,
        sampler=datasampler,
    )  # shuffle and sampler are mutually exclusive args

    #print(f'len(dataloader): {len(dataloader)}')
    return dataloader



In [ ]:
#cfg = {
#    "num_workers": 8,
#    "fold": 0,
#    "total_folds": 5,
#    "mean": "(0.485, 0.456, 0.406)",
#    "std": "(0.229, 0.224, 0.225)",
#    "size": 300,
#    'folder': '',
#    #"data_folder": "../data/npy_files/bgcc300",
#    "data_folder": "../data/train_images/",
#    "df_path": './data/train.csv',
#    "num_samples": "None",
#    "class_weights": "None",
#    "batch_size": {"train": 1, "val": 1},
#    "he_sampling": False, 
#    "tc_dups": True,
#    "dups_wsd": "../data/npy_files/good_duplicates.npy",
#    "bad_idx": "../data/npy_files/all_bad_duplicates.npy",
#    "sample": False,
#    "messidor_in_train":False,
#    "cw_sampling": False,
#}
import yaml
filepath = '../configs/218_efficientnet-b5_f1_o6cc2.yaml'
with open(filepath, 'r') as stream:
    try:
        cfg = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

dataloaders = {phase: provider(phase, cfg) for phase in ["train", "val"]}

In [ ]:
dataloaders['train'].dataset.transform

In [ ]:
files = []
for batch in dataloaders['train']:
    files.append(batch[0][0])

In [ ]:
val_files = []
for batch in dataloaders['val']:
    val_files.append(batch[0][0])

In [ ]:
for f in val_files:
    if f in files:
        print(f)

In [ ]:
duplicates = np.load('../data/duplicates.npy') # pairs which have same hash values, irrespective of their diagnosis

In [ ]:
for dups in duplicates:
    if (dups[0] in files and dups[1] in val_files) or (dups[0] in val_files and dups[1] in files):
        print(dups)
    # kucch to 3-3 files wale dups bhi hain

##### this shows the sanctity of val set is not maintained

### analyse augmentations 

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomContrast, RandomBrightness, Flip, OneOf, Compose, RandomGamma, ElasticTransform, ChannelShuffle,RGBShift, Rotate,
    RandomBrightnessContrast, CenterCrop, IAACropAndPad
)

In [ ]:
def strong_aug(p=1):
    return Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=.1),
            Blur(blur_limit=3, p=.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomContrast(),
            RandomBrightness(),
        ], p=0.3),
        #HueSaturationValue(p=0.3),
    ], p=p)

In [ ]:
# img = np.load('../data/npy_files/bgcc300/00cc2b75cddd.npy')
path = '../data/all_images/00cc2b75cddd.png'
size = 512
img = PP1(path, size)
print(img.dtype, img.shape)
plt.imshow(img);

In [ ]:
path = '../data/all_images/00cc2b75cddd.png'
img = Image.open(path)
img = np.array(img)
w, h, _ = img.shape
ratio = random.choice([0.75, 0.8, 0.85, 0.9])
print(ratio)
aug = CenterCrop(int(w*ratio), int(h*ratio), p=1)
aug_img = aug(image=img)['image']
plt.figure()
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.imshow(aug_img);

In [ ]:
aug = Transpose(p=1)
aug_img = aug(image=img)['image']
plt.imshow(aug_img)

In [ ]:
aug = Flip(p=1)
aug_img = aug(image=img)['image']
plt.imshow(aug_img)

In [ ]:
aug = ShiftScaleRotate(shift_limit=0, scale_limit=(-0.1, 0.3), rotate_limit=0, p=1)
aug_img = aug(image=img)['image']
plt.imshow(aug_img)

In [ ]:
aug = ShiftScaleRotate(shift_limit=0, scale_limit=0, rotate_limit=180, p=1)
aug_img = aug(image=img)['image']
plt.imshow(aug_img);

In [ ]:
aug = ShiftScaleRotate(shift_limit=0.1, scale_limit=0, rotate_limit=0, p=1)
aug_img = aug(image=img)['image']
plt.imshow(aug_img);

In [ ]:
aug = RandomBrightnessContrast(0.1, 0.1, 1)
aug_img = aug(image=img)['image']
plt.imshow(aug_img);

In [ ]:
def get_transforms(phase, cfg):
    size = cfg['size']
    mean = eval(cfg['mean'])
    std = eval(cfg['std'])
    list_transforms = [
        # albumentations.Resize(size, size) # now doing this in __getitem__()
    ]
    if phase == "train":
#         pass
        #list_transforms.extend(strong_aug())
        list_transforms.extend(
            [
                albumentations.Transpose(p=0.5),
                albumentations.Flip(p=0.5),
                albumentations.ShiftScaleRotate(shift_limit=0, rotate_limit=120, p=0.5, border_mode=cv2.BORDER_CONSTANT),
                albumentations.RandomBrightnessContrast(p=0.2),
            ]
        )

    list_transforms.extend(
        [
        albumentations.Normalize(mean=mean, std=std, p=1),    
#       albumentations.Resize(size, size), # because RandomScale is applied
        AT.ToTensor(),
        ]
    )
    return albumentations.Compose(list_transforms)

### Dataloader analysis

In [ ]:
import yaml
from preprocessing import *

filepath = '../configs/318_effnet-b5_f1_pord.yaml'
with open(filepath, 'r') as stream:
    try:
        cfg = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
cfg["num_workers"] = 1
cfg["batch_size"]["train"] = 1
cfg["batch_size"]["val"] = 1
dataloaders = {phase: provider(phase, cfg) for phase in ["train", "val"]}

In [ ]:
phase = "train"
dataloader = dataloaders[phase]
#dataloader.dataset.transform = get_transforms(phase, cfg)

In [ ]:
dataloader.dataset.transform.transforms.transforms

In [ ]:
def unnormalize(img):
    mean = torch.as_tensor([0.485, 0.456, 0.406])
    std = torch.as_tensor([0.229, 0.224, 0.225])
    #return (img * std) + mean
    for t, m, s in zip(img, mean, std):    
        t.mul_(s).add_(m)
    #img = (img * 255)
    return img

In [ ]:
# plot some random training images with their ground truth labels
batch = next(iter(dataloader))
fnames, images, labels = batch
plt.figure(figsize=(15, 15))
# i = random.choice(range(len(images)))
# print(i, len(images))
i = 0 # as the batch size is 1
img = images[i].clone()
img = unnormalize(img)
img = img.permute(1, 2, 0)
plt.subplot(1, 2, 1)
plt.imshow(np.array(img * 255, dtype='uint8'));
plt.title(torch.argmax(labels[i]).item())
plt.subplot(1, 2, 2)
org_img = Image.open(f'../data/all_images/{fnames[i]}')
# org_img = aug_6(f'../data/all_images/{fnames[i]}')
#plt.imshow(mpimg.imread(f'../data/train_images/{fnames[i]}.png')); # original image
plt.imshow(org_img); # original image
plt.show(); 

In [ ]:
path = '../data/train_images/0024cdab0c1e.png'
image = id_to_image(path,
        resize=True,
        size=300,
        augmentation=True,
        subtract_median=True,
        clahe_green=True)
plt.imshow(image)#[:, :, 0])

In [ ]:
transforms = Compose([
    #Transpose(p=0.5),
    #Flip(p=0.5),
    #ShiftScaleRotate(
    #    shift_limit=0,
    #    scale_limit=0.1,
    #    rotate_limit=180,
    #    p=1,
    #    #border_mode=cv2.BORDER_CONSTANT
    #),
    RandomBrightnessContrast(p=1),
])

In [ ]:
aug_img = transforms(image=image)["image"]
plt.imshow(aug_img)

In [ ]:
img.max(), img.min()

In [ ]:
labels[i].item()

In [ ]:
%matplotlib inline

In [ ]:
img = np.load('../data/train_images/npy_rgb/a00b4cb250a7.npy')

In [ ]:
plt.imshow(img)

In [ ]:
np.min(img), np.max(img)

In [ ]:
albumentations.Normalize # Divide pixel values by 255, subtract mean per channel and divide by std per channel

In [ ]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
norm = albumentations.Normalize(mean=mean, std=std, p=1)
norm_img = norm(image=img)['image']

In [ ]:
np.min(norm_img), np.max(norm_img), norm_img[0, 0, 0]

In [ ]:
norm_img.dtype

In [ ]:
albumentations.torch.ToTensor? # convert image and mask to torch.Tensor and divide by 255 if image or mask are `uint8` type, use with care look into sources

In [ ]:
albumentations.Normalize?

In [ ]:
mean = torch.as_tensor([0.485, 0.456, 0.406])
std = torch.as_tensor([0.229, 0.224, 0.225])
#return (img * std) + mean
#return img.mul_(std).add_(mean)
labels[i].item()

In [ ]:
mean, std, mean.shape, std.shape

### sampling

In [ ]:
def resampled(df, count_dict):
    ''' resample from df with replace=False'''
    def sample(obj):  # [5]
        return obj.sample(n=count_dict[obj.name], replace=False, random_state=69)
    sampled_df = df.groupby('diagnosis').apply(sample).reset_index(drop=True)
    return sampled_df


In [ ]:
sample_dict = {
    0: 1900, 
    2: 1900, 
    1: 1900, 
    3: 1900, 
    4: 1900
} 
df_old = pd.read_csv('../data/2015.csv')
sampled_df_old = resampled(df_old, sample_dict)


In [ ]:
sampled_df_old['diagnosis'].value_counts().plot(kind='bar')

In [ ]:
sampled_df_old.head()

In [ ]:
sampled_df_old.apply(lambda x: print(f'cp all_images/{x[0]} sampled/class_{x[1]}'), axis=1); # expand this, copy with keyboard ctrl+shift whatever, paste in a bash file, run it

In [ ]:
path = '../data/all_images/000c1434d8d7.png'
image = Image.open(path)

In [ ]:
image

In [ ]:
plt.imshow(np.array(image))

In [ ]:
import jpeg4py as jpeg
image = jpeg.JPEG(path).decode()


In [ ]:
path[-4:]

In [ ]:
plt.imshow(image)

### CLAHE

In [ ]:
# read, bgr->lab apply clahe, lab-rbg, median filtering
plt.figure(figsize=(20, 40))
bgr = cv2.imread('../data/all_images/000c1434d8d7.png')
bgr = crop_image(bgr, tol=10)
plt.subplot(1, 3, 1)

plt.imshow(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB));
# k = np.max(bgr.shape)//30*2+1 # kernel size: next odd number to (largest dim of the image) / 10
# bg = cv2.medianBlur(bgr, k)
# bgr = cv2.addWeighted (bgr, 4, bg, -4, 128)
# bg = cv2.GaussianBlur(bgr, (0, 0), sigmaX=10)
# bgr = cv2.addWeighted (bgr, 4, bg, -4, 128)

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
lab[...,0] = clahe.apply(lab[...,0])
rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
plt.subplot(1, 3, 2)
plt.imshow(rgb);

# k = np.max(rgb.shape)//30*2+1 # kernel size: next odd number to (largest dim of the image) / 10
bg = cv2.GaussianBlur(rgb, (0, 0), sigmaX=10)
# bg = cv2.medianBlur(rgb, k) 
frgb = cv2.addWeighted (rgb, 4, bg, -4, 128)
# frgb = rgb
plt.subplot(1, 3, 3)
plt.imshow(frgb)
plt.show();

In [ ]:
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1)
plt.imshow(frgb[:, :, 0]) # RED
plt.subplot(1, 3, 2)
plt.imshow(frgb[:, :, 1]) # GREEN
plt.subplot(1, 3, 3)
plt.imshow(frgb[:, :, 2]) # BLUE
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1)
plt.imshow(frgb[:, :, 0]) # RED
plt.subplot(1, 3, 2)
plt.imshow(frgb[:, :, 1]) # GREEN
plt.subplot(1, 3, 3)
plt.imshow(frgb[:, :, 2]) # BLUE
plt.show()

read, lab clahe, median is better than read, median, then lab clahe

In [ ]:
# read, bgr->lab apply clahe, lab-rbg, gaussian filtering
plt.figure(figsize=(20, 40))
bgr = cv2.imread('../data/all_images/000c1434d8d7.png')
bgr = crop_image(bgr, tol=10)
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB));
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
lab[...,0] = clahe.apply(lab[...,0])
rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
plt.subplot(1, 3, 2)
plt.imshow(rgb);

bg = cv2.GaussianBlur(rgb, (0, 0), 10)
frgb = cv2.addWeighted (rgb, 4, bg, -4, 128)
plt.subplot(1, 3, 3)
plt.imshow(frgb)
plt.show();

In [ ]:
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1)
plt.imshow(frgb[:, :, 0]) # RED
plt.subplot(1, 3, 2)
plt.imshow(frgb[:, :, 1]) # GREEN
plt.subplot(1, 3, 3)
plt.imshow(frgb[:, :, 2]) # BLUE
plt.show()


In [ ]:
# read, apply clahe to each channel, median filtering
plt.figure(figsize=(20, 40))
bgr = cv2.imread('../data/all_images/000c1434d8d7.png')
bgr = crop_image(bgr, tol=10)
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB));
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
bgr[...,0] = clahe.apply(bgr[...,0])
bgr[...,1] = clahe.apply(bgr[...,1])
bgr[...,2] = clahe.apply(bgr[...,2])
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
plt.subplot(1, 3, 2)
plt.imshow(rgb);

k = np.max(rgb.shape)//20*2+1 # kernel size: next odd number to (largest dim of the image) / 10
bg = cv2.medianBlur(rgb, k)
frgb = cv2.addWeighted (rgb, 4, bg, -4, 128)
plt.subplot(1, 3, 3)
plt.imshow(frgb)
plt.show();

In [ ]:
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1)
plt.imshow(frgb[:, :, 0]) # RED
plt.subplot(1, 3, 2)
plt.imshow(frgb[:, :, 1]) # GREEN
plt.subplot(1, 3, 3)
plt.imshow(frgb[:, :, 2]) # BLUE
plt.show()

# clahe to blue channel individually, ruins it 

In [ ]:
# read, apply clahe to all channels, gaussian filtering
plt.figure(figsize=(20, 40))
bgr = cv2.imread('../data/all_images/000c1434d8d7.png')
bgr = crop_image(bgr, tol=10)
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB));
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
bgr[...,0] = clahe.apply(bgr[...,0])
bgr[...,1] = clahe.apply(bgr[...,1])
bgr[...,2] = clahe.apply(bgr[...,2])
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
plt.subplot(1, 3, 2)
plt.imshow(rgb);

bg = cv2.GaussianBlur(rgb, (0, 0), 10) #sigmaX=10
frgb = cv2.addWeighted (rgb, 4, bg, -4, 128)
plt.subplot(1, 3, 3)
plt.imshow(frgb)
plt.show();

In [ ]:
plt.figure(figsize=(20, 40))
bgr = cv2.imread('../data/all_images/000c1434d8d7.png')
bgr = crop_image(bgr, tol=10)
bgr = cv2.resize(bgr, (256, 256))
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB));
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
lab[...,0] = clahe.apply(lab[...,0])
rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
plt.subplot(1, 3, 2)
plt.imshow(rgb);
k = np.max(rgb.shape)//20*2+1 # kernel size: next odd number to (largest dim of the image) / 10
bg = cv2.medianBlur(rgb, k)
frgb = cv2.addWeighted (rgb, 4, bg, -4, 128)
# frgb = cv2.resize(frgb, (256, 256))
plt.subplot(1, 3, 3)
plt.imshow(frgb)
plt.show();
# cv2.imwrite('before.png',frgb)

In [ ]:
idxs = [  23,   45,   82,  120,  126,  166,  192,  198,  200,  230,  325,
        360,  378,  409,  538,  572,  671,  699,  756,  771,  889, 1024,
       1183, 1280, 1393, 1455, 1469, 1540, 1546, 1547, 1605, 1629, 1643,
       1724, 1753, 1814, 1833, 1850, 1918, 2034, 2400, 2403, 2576, 2637,
       2680, 2782, 2793, 2794, 3006, 3276, 3357, 3370]

In [ ]:
df = pd.read_csv('../data/train.csv')
# for idx, row in tqdm(df.iterrows()):
for idx in idxs:
    try:
        path = '../data/all_images/' + df.iloc[idx]['id_code']# + '.png'
        bgr = cv2.imread(path)
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        print(rgb.shape)
#         plt.imshow(rgb); plt.show()
        rgb = crop_image(rgb, tol=9)
        print(rgb.shape, '\n')
#         plt.imshow(rgb); plt.show()
        k = np.max(rgb.shape)//20*2+1 # kernel size: next odd number to (largest dim of the image) / 10
        bg = cv2.medianBlur(rgb, k)
    except Exception as e:
        print(idx, e)
        break
# exceptions start after 35138

In [ ]:
plt.figure(figsize=(20, 40))
bgr = cv2.imread('test.jpeg')
bgr = crop_image(bgr, tol=10)
bgr = cv2.resize(bgr, (256, 256))
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB));
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
lab[...,0] = clahe.apply(lab[...,0])
rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
plt.subplot(1, 3, 2)
plt.imshow(rgb);
print(rgb.shape)
k = np.min(rgb.shape[:2])//20*2+1 # kernel size: next odd number to (largest dim of the image) / 10
print(k)
bg = cv2.medianBlur(rgb, k)
frgb = cv2.addWeighted (rgb, 4, bg, -4, 128)
# frgb = cv2.resize(frgb, (256, 256))
plt.subplot(1, 3, 3)
plt.imshow(frgb)
plt.show();
# cv2.imwrite('before.png',frgb)

In [ ]:
path = '../data/all_images/' + df.iloc[idx]['id_code']# + '.png'
bgr = cv2.imread(path)
print(bgr.shape)
bgr = crop_image(bgr, tol=7)
print(bgr.shape, '\n')
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
plt.imshow(rgb)
k = np.max(rgb.shape)//22*2+1 # kernel size: next odd number to (largest dim of the image) / 10
bg = cv2.medianBlur(rgb, k)

In [ ]:
plt.imshow(bgr)

In [ ]:
path = '../data/all_images/e9ff9352ccb3.png'
bgr = cv2.imread(path)
bgr = crop_image(bgr, tol=10)
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
k = np.max(rgb.shape)//20*2+1 # kernel size: next odd number to (largest dim of the image) / 10
print(k)
# k=351
bg = cv2.medianBlur(rgb, k)
rgb =cv2.addWeighted (rgb, 4, bg, -4, 128)
rgb = cv2.resize(rgb, (256, 256))
plt.imshow(rgb)

In [ ]:
#https://github.com/opencv/opencv/blob/master/modules/imgproc/src/median_blur.simd.hpp#L337

###  GREEN channels

In [ ]:
# https://www.kaggle.com/c/aptos2019-blindness-detection/discussion/102613#591316
def toCLAHEgreen(img):  
    clipLimit=2.0 
    tileGridSize=(8, 8)  
    img = np.array(img)     
    green_channel = img[:, :, 2]    
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    cla = clahe.apply(green_channel) 
    cla=clahe.apply(cla)
    return cla

In [ ]:
bgr = cv2.imread('../data/all_images/000c1434d8d7.png')
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
gimg = toCLAHEgreen(rgb)
plt.imshow(gimg)

In [ ]:
gimg.shape
#gimg = gimg.reshape(456, 456, 1)
gimg = np.expand_dims(gimg, -1)
img = np.repeat(gimg, 3, -1)

In [ ]:
img.shape

### Median filter

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
IMAGE_SIZE = 256

In [ ]:
def info_image(im):
    # Compute the center (cx, cy) and radius of the eye
    cy = im.shape[0]//2
    midline = im[cy,:]
    midline = np.where(midline>midline.mean()/3)[0]
    if len(midline)>im.shape[1]//2:
        x_start, x_end = np.min(midline), np.max(midline)
    else: # This actually rarely happens p~1/10000
        x_start, x_end = im.shape[1]//10, 9*im.shape[1]//10
    cx = (x_start + x_end)/2
    r = (x_end - x_start)/2
    return cx, cy, r

def resize_image(im, augmentation=True):
    # Crops, resizes and potentially augments the image to IMAGE_SIZE
    cx, cy, r = info_image(im)
    scaling = IMAGE_SIZE/(2*r)
    rotation = 0
    if augmentation:
        scaling *= 1 + 0.3 * (np.random.rand()-0.5)
        rotation = 360 * np.random.rand()
    M = cv2.getRotationMatrix2D((cx,cy), rotation, scaling)
    M[0,2] -= cx - IMAGE_SIZE/2
    M[1,2] -= cy - IMAGE_SIZE/2
    return cv2.warpAffine(im,M,(IMAGE_SIZE,IMAGE_SIZE)) # This is the most important line

def subtract_median_bg_image(im):
    k = np.max(im.shape)//20*2+1
    bg = cv2.medianBlur(im, k)
    return cv2.addWeighted (im, 4, bg, -4, 128)

def subtract_gaussian_bg_image(im):
    k = np.max(im.shape)/10
    bg = cv2.GaussianBlur(im ,(0,0) ,k)
    return cv2.addWeighted (im, 4, bg, -4, 128)

def toCLAHEgreen(img):
    clipLimit=2.0
    tileGridSize=(8, 8)
    img = np.array(img)
    green_channel = img[:, :, 1]
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    cla = clahe.apply(green_channel)
    cla=clahe.apply(cla)
    return cla

def apply_clahe(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    lab[...,0] = clahe.apply(lab[...,0])
    rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
    return rgb

def id_to_image(id_code, resize=True, augmentation=False, clahe=False, subtract_gaussian=False, subtract_median=False, clahe_green=False):
    #path = '../data/all_images/{}'.format(id_code)
    path = '../data/test_images/{}.png'.format(id_code)
    im = cv2.imread(path)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    if resize_image:
        im = resize_image(im, augmentation)
    if clahe:
        im = apply_clahe(im)
    if clahe_green:
        im = toCLAHEgreen(im)
    if subtract_gaussian:
        im = subtract_gaussian_bg_image(im)
    if subtract_median:
        im = subtract_median_bg_image(im)
    return im

In [ ]:
%%time
df_train = pd.read_csv('../data/sample_submission.csv')
# df_train = pd.read_csv('../data/train.csv')
fig = plt.figure(figsize=(25, 16))
SEED = np.random.randint(0,100)

def plot_col(col, id2im, n_cols=6):
    for class_id in range(0, 5):
        class_id = 0
        for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(1, random_state=SEED).iterrows()):
            ax = fig.add_subplot(5, n_cols, class_id * n_cols + i + col, xticks=[], yticks=[])
            im = id2im(row['id_code'])
            plt.imshow(im)
            ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

# display normal image of each class
plot_col(1, lambda x: id_to_image(x))

# display normal image of each class
plot_col(4, lambda x: id_to_image(x, augmentation=True))

# display normal image of each class
plot_col(2, lambda x: id_to_image(x, subtract_gaussian=True))

# display normal image of each class
# plot_col(5, lambda x: id_to_image(x, subtract_gaussian=True, augmentation=True))
plot_col(3, lambda x: id_to_image(x, clahe=True))

# display normal image of each class
# plot_col(3, lambda x: id_to_image(x, subtract_median=True))

# display normal image of each class
# plot_col(6, lambda x: id_to_image(x, subtract_median=True, augmentation=True))

The first column shows the normal images, the second columns shows the images processed with an approach similar to Ben Graham's approach.

The third column shows the images when a Median filter is used (instead of a Gaussian filter). A median filter is very effective in seperating small or narrow objects from a relatively continous background. As a result, the blood vessels and small objects are much more apparant in the third column compared to the first two. That may be useful (or not!). It is also relatively succesful in removing color from the image, which may make images shot with different cameras more comparable (or removes potentially usefull information!). Put it in the comments if you get better/worse results with the Median filter.

The last three columns just show random rotations/scalings. You can use the code to implement your own ImageGenerator and generate augmented images on the fly.

In [ ]:
df = pd.read_csv('../data/train.csv')
idx = 0
img = id_to_image(df.iloc[idx]["id_code"], subtract_median=True)
print(img.shape)
plt.imshow(img)

In [ ]:
img = id_to_image(df.iloc[idx]["id_code"], clahe_green=True)
plt.imshow(img)

In [ ]:
img = id_to_image(df.iloc[idx]["id_code"], subtract_median=True, clahe_green=True)
plt.imshow(img)

In [ ]:
img = id_to_image(df.iloc[idx]["id_code"], subtract_median=True, clahe_green=True)
plt.imshow(img)

In [ ]:
img.shape

### pretrained models testing

In [ ]:
import pretrainedmodels
pretrainedmodels.model_names

In [ ]:
pretrainedmodels.pretrained_settings['inceptionresnetv2']

In [ ]:
summary(model2, input_size=(3, 224, 224), device="cpu")

In [ ]:
import torch
model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x16d_wsl')

In [ ]:
model.fc

In [ ]:
model.avgpool

In [ ]:
model.children()

In [ ]:
blocks = list(model.children())

In [ ]:
backbone = nn.Sequential(
    *list(blocks[:-2])
) # upto AdaptiveAvgPool2d

In [ ]:
backbone

In [ ]:
nn.AvgPool2d?

In [ ]:
# model_name='se_resnext50_32x4d'
model_name= "resnext101_32x4d"#'nasnetamobile'
model_name = "resnet50"
#model_name = "densenet121"

model = pretrainedmodels.__dict__[model_name](num_classes=1000)

In [ ]:
#list(model.children())
model

In [ ]:
#model.features

In [ ]:
# blocks = list(model.children())
# # blocks[:-3]
# model

In [ ]:
blocks = list(model.children())
head = torch.nn.Sequential(*list(model.children())[:-3])
# model._modules['layer0'] = head
#model._modules['avg_pool'] = torch.nn.AvgPool2d(kernel_size=5, stride=1, padding=0) 
#model._modules['avg_pool'] = torch.nn.Sequential(*[torch.nn.ConstantPad2d((0,1,0,1), 0), torch.nn.AvgPool2d(kernel_size=7, stride=1, padding=0)])
# model._modules['avg_pool'] = torch.nn.Sequential(torch.nn.AdaptiveAvgPool2d((1, 1)))
# model._modules['last_linear'] = torch.nn.Linear(in_features=2048, out_features=1, bias=True)
# model._modules['last_layer'] = classifier

In [ ]:
torch.nn.AdaptiveAvgPool2d?

In [ ]:
#head.parameters()
#for block in blocks:
base = list(blocks[0].children())[:-1] + blocks[1:-2]
torch.nn.Sequential(*base)
torch.nn.Dropout?
#    print(block)
#    print('*'*100)

In [ ]:

torch.nn.Linear?

In [ ]:
#blocks[0] = head

In [ ]:
#blocks[-1] = torch.nn.Linear(in_features=2048, out_features=1, bias=True)

In [ ]:
#model = torch.nn.Sequential(*blocks)

In [ ]:
#model._modules

In [ ]:
img = torch.Tensor(1, 3, 224, 224)#.to('cuda')
#img = torch.Tensor(1, 3, 112, 112)
#img = torch.Tensor(1, 3, 96, 96)

In [ ]:
out = model(img)
features = model.features(img)
print(features.shape, out.shape)

In [ ]:
nn.AdaptiveAvgPool2d((1, 1))(features).shape

In [ ]:
x = None
for param in model.features.children():
    
    print(param)
    if x is None:
        x = param(img)
    else:
        x = param(x)
    print(x.shape)

In [ ]:
# summary(model, input_size=(3, 96, 96))
# summary(model, input_size=(3, 112, 112))
summary(model.features, input_size=(3, 224, 224))

In [ ]:
torch.nn.functional.max_pool2d
torch.nn.functional.adaptive_avg_pool2d?

In [ ]:
# torch.nn.Dropout?

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)




In [ ]:
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained="imagenet")
backbone = model.features
classifier = nn.Sequential(
        nn.AdaptiveAvgPool2d((1, 1)),
        Flatten(),
        nn.Dropout(0.5),
        nn.Linear(in_features=1056, out_features=1, bias=True)
        )



In [ ]:
img = torch.Tensor(1, 3, 112, 112)
features = backbone(img)


In [ ]:
model.parameters()

In [ ]:
 
from albumentations import *

In [ ]:
train_aug = Compose([
    Resize(224, 224),
    RandomRotate90(p=0.5),
    Transpose(p=0.5),
    Flip(p=0.5),
    OneOf([CLAHE(clip_limit=2), IAASharpen(), IAAEmboss(), RandomBrightnessContrast(), JpegCompression(), Blur(), GaussNoise() ], p=0.5), 
    HueSaturationValue(p=0.5), 
    ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit=45, p=0.5),
    Normalize(p=1)
])
val_aug = Compose([
    Normalize(p=1)
])


In [ ]:
ShiftScaleRotate?

In [ ]:
Normalize?

In [ ]:
train_aug?

In [ ]:
flip = Flip(p=1)

In [ ]:
flip.apply?

### AUC ROC

In [ ]:
# roc curve and auc
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, weights=[1,1], random_state=1)
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# fit a model
model = KNeighborsClassifier(n_neighbors=3)
model.fit(trainX, trainy)
# predict probabilities
probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
probs = probs[:, 1]


In [ ]:
testy.shape, probs.shape

In [ ]:
fpr, tpr, thresholds

In [ ]:
# calculate AUC
auc = roc_auc_score(testy, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(testy, probs)
# plot no skill
pyplot.plot([0, 1], [0, 1], linestyle='--')
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
pyplot.plot(fpr, tpr, marker='.')
# show the plot
pyplot.show()

In [ ]:
pyplot.savefig('lol.png')

In [ ]:
pyplot.close('all')

In [ ]:
import apex

In [ ]:
apex.amp.?

In [ ]:
import pandas as pd
df1 = pd.read_csv('weights/ensemble/11Mar_nasnetamobile_fold0_ckpt17.csv')
df2 = pd.read_csv('weights/ensemble/9Mar_se_resnext50_32x4d_v3_fold0_1_2_allckpt17.csv')
df3 = pd.read_csv('weights/ensemble/11Mar_nasnetamobile_v2_fold2_ckpt20.csv')
df= df1.copy()
df['label'] = (df1['label'] + df3['label'] + 3* df2['label'])/5

In [ ]:
df.head()

In [ ]:
df.to_csv('weights/ensemble/resnextfold012ckpt17+nasnetmobileckpt17+nasnetamobilefold2_ckpt20.csv', index=False)

### sampled val set

In [ ]:
sample_dict = {
        2:    0.647303,
        0:    0.185166,
        1:    0.070539,
        3:    0.058091,
        4:    0.038900
}


In [ ]:
count = {}
for i, j in sample_dict.items():
    count[i] = int(sample_dict[i] * 650)
count,  sum(count.values())

In [ ]:
vdf = pd.read_csv('../data/folds/val0-5.csv')
vdf.shape

In [ ]:
train_df = pd.read_csv('../data/train.csv')

In [ ]:
train_df.head()

In [ ]:
def sample(obj):  # [5]
    return obj.sample(n=count[obj.name], replace=False, random_state=69)
val_df = train_df.groupby('diagnosis').apply(sample).reset_index(drop=True)

In [ ]:
val_df.head()

In [ ]:
train_df.shape, val_df.shape

In [ ]:
train_df =train_df[~train_df['id_code'].isin(val_df.id_code.tolist())]

In [ ]:
train_df.shape, val_df.shape

In [ ]:
val_df.head()

In [ ]:
train_df = train_df[train_df.diagnosis != 3]

In [ ]:
train_df['diagnosis'] -= 1 

In [ ]:
train_df